# <center> POVM as a quantum version of the classifier </center>

In [95]:
import pennylane as qml
from pennylane import numpy as np

In [96]:
dev = qml.device('default.qubit', wires=2)

In [115]:
dev = qml.device('qiskit.aer', wires=2)
#dev = qml.device('qiskit.ibmq', wires=2, backend='')

ContextualVersionConflict: (scipy 1.7.1 (/Users/Mac_dk/anaconda3/lib/python3.7/site-packages), Requirement.parse('scipy<=1.6.1,>=1.4'), {'qiskit-aqua'})

## A module of the two-element povm.

In [98]:
def two_element_povm(params, wire0, wire1):
    #params 

    # Controlled-RY gate controlled by first qubit in |0> state
    qml.PauliX(wires=wire0)
    qml.CRY(params[0], wires=[wire0,wire1])
    qml.PauliX(wires=wire0)
    
    # Controlled-RY gate controlled by first qubit in |1> state
    qml.CRY(params[1], wires=[wire0,wire1])

    # Controlled-Rotation gate (arbitrary single-qubit unitary operator) controlled by 2nd qubit in |0> state
    qml.PauliX(wires=wire1)
    qml.CRot(params[2], params[3], params[4], wires=[wire1,wire0])
    qml.PauliX(wires=wire1)

    # # Controlled-Rotation gate (arbitrary single-qubit unitary operator) controlled by 2nd qubit in |1> state
    qml.CRot(params[5], params[6], params[7], wires=[wire1,wire0])

Drawing `two_element_povm` module

In [99]:
@qml.qnode(dev)
def povm_circuit(params):
    two_element_povm(params, 0, 1)
    return qml.expval(qml.Identity(0) @ qml.PauliZ(1))

# initial parameters
params = np.random.random([8])
povm_circuit(params)

print("Drawing a two-element POVM circuit : ")
print(povm_circuit.draw())

Drawing a two-element POVM circuit : 
 0: ──X──╭C──────────X──╭C─────────────╭Rot(0.464, 0.499, 0.147)─────╭Rot(0.395, 0.911, 0.718)──╭┤ ⟨I ⊗ Z⟩ 
 1: ─────╰RY(0.942)─────╰RY(0.874)──X──╰C─────────────────────────X──╰C─────────────────────────╰┤ ⟨I ⊗ Z⟩ 



---

## Preparing two arbitrary pure states

In [100]:
def state_preparation(params):
    return

---

## Minimum Error Discrimination

Preparation: $\{q_i,\hat{\rho}_i\}^{N}_{i=1}$ $\longleftarrow$ $\hat{\rho} = \sum_i q_i\hat{\rho}_i$. <br>

POVMs: $\{E_k\}_{k=1}^{L}$ with $\sum_{k=1}^{L} E_k = I$.

Q. Why "$N=L$" is optimal?

In general, for a state $\hat{\rho}_i$ generated in preparation, a detection event on $\hat{E}_k$ happens with probability $ p(k|i) = tr [\hat{E}_k\hat{\rho_i}] $.

$$
p_{success} = \sum_i q_i p(i|i)
$$
$$
p_{guess} = \max_{E} \sum_i q_i p(i|i)
$$
$$
p_{error} = 1 - p_{guess}
$$

For two-element POVMs $\{\hat{E}_1, \hat{E}_2\}$ on two different state prepared as $\{(q_1,\hat{\rho}_1), (q_2,\hat{\rho}_2)\}$, 
$$
p_{error} = 1 - p_{guess} = 1-\max_{\vec{\theta}}\left(q_o\text{Tr}[\hat{E}_0\hat{\rho}_0] + q_1\text{Tr}[\hat{E}_1\hat{\rho}_1] \right)
$$

---

## Classification of two different states by using POVM

$|\Psi^0_i\rangle := |\psi_0\rangle|0\rangle$ 
$\longrightarrow$ POVM $\longrightarrow$ 
$|\Psi^0_f\rangle := \left(\hat{K}_0|\psi_0\rangle\right)|0\rangle + \left(\hat{K}_1|\psi_0\rangle\right)|1\rangle$

$$
\text{Tr}[\hat{\rho}_0\hat{E}_0] = \langle\psi_0|\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle \langle 0 | 0 \rangle = \langle\psi_0|\langle 0|\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle|0\rangle\\
=\frac{1}{2}\left(\langle\psi_0|\langle 0|\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle|0\rangle + \langle\psi_0|\langle 1|\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle|1\rangle\right)\\
+\frac{1}{2}\left(\langle\psi_0|\langle 0|\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle|0\rangle - \langle\psi_0|\langle 1|\hat{K}^\dagger_0\hat{K}_0|\psi_0\rangle|1\rangle\right) \\ 
\frac{1}{2} \left(1 + \langle\Psi_f|I\otimes Z|\Psi_f\rangle\right)
$$
Similarly, $\text{Tr}[\hat{\rho}_0\hat{E}_1] = \frac{1}{2}\left(1 - \langle\Psi_f|I\otimes Z|\Psi_f\rangle\right)$.

Find **the optimal POVMs** for discriminating two different states:
$$
p_{error} = 1- (-1)\min_{\vec{\theta}}\left(-q_o\text{Tr}[\hat{E}_0\hat{\rho}_0] - q_1\text{Tr}[\hat{E}_1\hat{\rho}_1]\right)\\
=\min_{\vec{\theta}}\left[\frac{1}{2} - \frac{q_0}{2}\langle\Psi^0_f|I\otimes Z|\Psi^0_f\rangle + \frac{q_1}{2}\langle\Psi^1_f|I\otimes Z|\Psi^1_f\rangle\right]
$$

In [101]:
dev = qml.device('default.qubit', wires=2)

$ \langle\Psi^0_f|I\otimes Z|\Psi^0_f\rangle $

In [113]:
@qml.qnode(dev)
def circuit_povm_expvalIZpsi0(params):
    # Initial state: |0>
    qml.Hadamard(wires=0) # |+>

    # arbitrary rotation
    qml.Rot(params[0], params[1], params[2], wires=0)

    # two-element POVM
    two_element_povm(params[3:], 0, 1)

    return qml.expval(qml.Identity(0) @ qml.PauliZ(1)) 

$ \langle\Psi^1_f|I\otimes Z|\Psi^1_f\rangle $

In [103]:
@qml.qnode(dev)
def circuit_povm_expvalIZpsi1(params):
    # Initial state: X|0> = |1>
    qml.PauliX(wires=0)
    qml.Hadamard(wires=0) # |->

    # arbitrary rotation
    qml.Rot(params[0], params[1], params[2], wires=0)

    # two-element POVM
    two_element_povm(params[3:], 0, 1)

    return qml.expval(qml.Identity(0) @ qml.PauliZ(1))

Cost function : $C(\vec{\theta}) = \frac{1}{2}\left(1 - q_0\langle\Psi^0_f|I\otimes Z|\Psi^0_f\rangle + q_1\langle\Psi^1_f|I\otimes Z|\Psi^1_f\rangle\right)$

In [104]:
def cost(x):
    K0psi0 = circuit_povm_expvalIZpsi0(x)
    K1psi1 = circuit_povm_expvalIZpsi1(x)
    return (1/2) * (1 - (1/2)*K0psi0 + (1/2)*K1psi1)


### Optimization of POVM: solving $\min_{\vec{\theta}}C(\vec{\theta})$

In [114]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 200

# set the initial parameter values
params = 2 * np.pi * np.random.random([11])
print("cost(init_params) =", cost(params))

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i+1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

cost(init_params) = 0.6936881499191591
Cost after step     5:  0.5925137
Cost after step    10:  0.5318706
Cost after step    15:  0.4917141
Cost after step    20:  0.4491884
Cost after step    25:  0.3823426
Cost after step    30:  0.2755206
Cost after step    35:  0.1522323
Cost after step    40:  0.0657218
Cost after step    45:  0.0248550
Cost after step    50:  0.0089242
Cost after step    55:  0.0031446
Cost after step    60:  0.0011006
Cost after step    65:  0.0003843
Cost after step    70:  0.0001341
Cost after step    75:  0.0000467
Cost after step    80:  0.0000163
Cost after step    85:  0.0000057
Cost after step    90:  0.0000020
Cost after step    95:  0.0000007
Cost after step   100:  0.0000002
Cost after step   105:  0.0000001
Cost after step   110:  0.0000000
Cost after step   115:  0.0000000
Cost after step   120:  0.0000000
Cost after step   125:  0.0000000
Cost after step   130:  0.0000000
Cost after step   135:  0.0000000
Cost after step   140:  0.0000000
Cost afte

In [106]:
print("Drawing a circuit of povm_expvalIZ_psi0 : ")
print(circuit_povm_expvalIZpsi0.draw())
print("Drawing a circuit of povm_expvalIZ_psi1 : ")
print(circuit_povm_expvalIZpsi1.draw())

Drawing a circuit of povm_expvalIZ_psi0 : 
 0: ──Rot(3.14, 3.93, 0.0251)──X──╭C─────────X──╭C────────────╭Rot(3.47, 1.26, 3.06)─────╭Rot(1.52, 2.48, 1.6)──╭┤ ⟨I ⊗ Z⟩ 
 1: ──────────────────────────────╰RY(3.14)─────╰RY(6.28)──X──╰C──────────────────────X──╰C─────────────────────╰┤ ⟨I ⊗ Z⟩ 

Drawing a circuit of povm_expvalIZ_psi1 : 
 0: ──X──H──Rot(3.14, 3.93, 0.0251)──X──╭C─────────X──╭C────────────╭Rot(3.47, 1.26, 3.06)─────╭Rot(1.52, 2.48, 1.6)──╭┤ ⟨I ⊗ Z⟩ 
 1: ────────────────────────────────────╰RY(3.14)─────╰RY(6.28)──X──╰C──────────────────────X──╰C─────────────────────╰┤ ⟨I ⊗ Z⟩ 



## Calculating Kraus Operators or POVMs

### Arbitrary rotation of the single qubit
### $$ U = R_z(\beta)R_y(\gamma)R_z(\delta) = \begin{bmatrix} e^{-i(\delta+\beta)/2}\cos(\gamma/2) & -e^{i(\delta-\beta)/2}\sin(\gamma/2) \\ e^{-i(\delta-\beta)/2}\sin(\gamma/2) & e^{i(\delta+\beta)/2}\cos(\gamma/2) \end{bmatrix}$$

### Y-Rotation of the single qubit
### $$ R_y(\phi) = e^{-i\phi\sigma_y/2} = \begin{bmatrix} \cos\phi/2 & -\sin\phi/2 \\ \sin\phi/2 & \cos\phi/2 \end{bmatrix} $$

In [107]:
def unitaries_in_povm(params):
    U = qml.Rot(params[0], params[1], params[2], wires=2).matrix
    Ry0 = qml.RY(params[3], wires=2).matrix
    Ry1 = qml.RY(params[4], wires=2).matrix
    V0 = qml.Rot(params[5], params[6], params[7], wires=2).matrix
    V1 = qml.Rot(params[8], params[9], params[10], wires=2).matrix

    return U, Ry0, Ry1, V0, V1

U, Ry0, Ry1, V0, V1 = unitaries_in_povm(params)

$$
\hat{K}_1 = V_1D_1U \\
\hat{K}_2 = V_2D_2U 
$$

In [108]:
def kraus_op(params):
    U, _, _, V0, V1 = unitaries_in_povm(params)
    D0 = np.diag([np.cos(params[3]/2), np.cos(params[4]/2)])
    D1 = np.diag([np.sin(params[3]/2), np.sin(params[4]/2)])
    K0 = np.dot(np.dot(V0, D0), U)
    K1 = np.dot(np.dot(V1, D1), U)

    return K0, K1

K0, K1 = kraus_op(params)

$ \hat{K}^\dagger_1\hat{K}_1 + \hat{K}^\dagger_2\hat{K}_2 = I$

In [109]:
np.dot(K0.conj().T, K0) + np.dot(K1.conj().T, K1)

tensor([[ 1.00000000e+00+0.00000000e+00j,
         -5.55111512e-17-1.21430643e-17j],
        [-5.55111512e-17+1.21430643e-17j,
          1.00000000e+00+0.00000000e+00j]], requires_grad=True)

$ \langle \psi_0 $

In [110]:
res = np.dot(K1, np.array([1,0]))
np.dot(res.conj(),res)

(0.1464466093903767+0j)

In [111]:
res = np.dot(K2, np.array([1/np.sqrt(2),-1/np.sqrt(2)]))
np.dot(res.conj(),res)

(0.9999999999855727+0j)

In [112]:
K1

tensor([[ 0.12477206-4.31158281e-04j, -0.30122637+1.04122018e-03j],
        [-0.00929095+3.61651815e-01j,  0.02243358-8.73104642e-01j]], requires_grad=True)